In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
ALL_BRAWLERS_URL = 'https://brawlstars.fandom.com/wiki/Category:Brawlers'
response = requests.get(ALL_BRAWLERS_URL)
soup = BeautifulSoup(response.content, 'html.parser')
brawlers = soup.find_all('a', {'class':'category-page__member-link'})

In [3]:
names_lst = []
for brawler in brawlers:
    name = brawler.text
    if 'Category:' in name or 'User blog' in name:
        continue
    names_lst.append(name)

In [4]:
brawlerDict = {}
for name in names_lst:
    BRAWLER_URL = f'https://brawlstars.fandom.com/wiki/{name}'
    response = requests.get(BRAWLER_URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Info
    brawlerInfo = {}
    data_sources = ['Rarity', 'Class','MovementSpeed','AttackRange','AttackBullets','Reload','AttackSuperCharge','AttackSpeed','AttackWidth','SuperRange','SuperSuperCharge','SuperSpeed']
    for data_source in data_sources:
        try:
            raw_value = soup.find_all('div', {'data-source':f'{data_source}'})[0].find('div', {'class': 'pi-data-value'}).get_text(separator=', ', strip = True)#.text.split('<br>')[0]#.strip()
            cleaned_value = re.split('\(|%|°|, |º|second', raw_value)[0].strip()
            if '-' in cleaned_value:
                cleaned_value = abs(float(cleaned_value.split('-')[0]) - float(cleaned_value.split('-')[1])) / 2
            brawlerInfo[f'{data_source}'] = cleaned_value
        except:
            brawlerInfo[f'{data_source}'] = float('nan')
    if name == 'Gray': brawlerInfo['SuperSpeed'] = float('nan')
    
    # Health
    healthDict = {}
    health = soup.find_all('td', {'data-source':'Health'})[1:]
    for i in range(len(health)):
        if i%2 == 0:
            healthDict[f'{health[i].text.strip()}'] = health[i+1].text.strip()
    brawlerInfo['Health'] = healthDict
    
    # Attack
    attackDict = {}
    attack = soup.find_all('td', {'data-source':'Attack'})[1:]
    for i in range(len(attack)):
        if i%2 == 0:
            attackDict[f'{attack[i].text.strip()}'] = attack[i+1].text.strip()
    brawlerInfo['Attack'] = attackDict
    
    brawlerDict[f'{name.upper()}'] = brawlerInfo
    print(name)
    

8-Bit
Amber
Ash
Barley
Bea
Belle
Bibi
Bo
Bonnie
Brock
Bull
Buster
Buzz
Byron
Carl
Chester
Colette
Colt
Crow
Darryl
Dynamike
Edgar
El Primo
Emz
Eve
Fang
Frank
Gale
Gene
Gray
Griff
Grom
Gus
Jacky
Janet
Jessie
Leon
Lola
Lou
Mandy
Max
Meg
Mortis
Mr. P
Nani
Nita
Otis
Pam
Penny
Piper
Poco
R-T
Rico
Rosa
Ruffs
Sam
Sandy
Shelly
Spike
Sprout
Squeak
Stu
Surge
Tara
Tick
Willow


In [5]:
brawlerDict

{'8-BIT': {'Rarity': 'Super Rare',
  'Class': 'Damage Dealer',
  'MovementSpeed': '580',
  'AttackRange': '10',
  'AttackBullets': '6',
  'Reload': '1.5',
  'AttackSuperCharge': '8.48',
  'AttackSpeed': '4500',
  'AttackWidth': '0.33',
  'SuperRange': '5',
  'SuperSuperCharge': nan,
  'SuperSpeed': '1196',
  'Health': {'1': '4800',
   '2': '5040',
   '3': '5280',
   '4': '5520',
   '5': '5760',
   '6': '6000',
   '7': '6240',
   '8': '6480',
   '9': '6720',
   '10': '6960',
   '11': '7200'},
  'Attack': {'1': '320',
   '2': '336',
   '3': '352',
   '4': '368',
   '5': '384',
   '6': '400',
   '7': '416',
   '8': '432',
   '9': '448',
   '10': '464',
   '11': '480'}},
 'AMBER': {'Rarity': 'Legendary',
  'Class': 'Controller',
  'MovementSpeed': '720',
  'AttackRange': '8.33',
  'AttackBullets': '40',
  'Reload': '0.22',
  'AttackSuperCharge': '4.55',
  'AttackSpeed': '3500',
  'AttackWidth': '1.33',
  'SuperRange': '7.33',
  'SuperSuperCharge': '6.36',
  'SuperSpeed': '1750',
  'Health'

In [6]:
import json
with open("./data/brawlerInfo.json", "w") as outfile:
    json.dump(brawlerDict, outfile)

## Exporting Data to S3

In [7]:
!aws s3 ls

2023-02-13 13:14:50 si699-mj199-bucket


In [8]:
import time
date_string = time.strftime("%Y%m%d")

import boto3
s3client = boto3.client('s3')
s3client.upload_file('./data/brawlerInfo.json',  'si699-mj199-bucket', f'data/{date_string}/brawlerInfo.json')

## Optional: Loading Data to AWS RDS (Database)

In [14]:
# brawler_info_df = pd.DataFrame(brawlerDict).T.iloc[:,:-2]
# brawler_info_df = brawler_info_df.reset_index()
# brawler_info_df = brawler_info_df.rename(columns = {'index':'name'})
# brawler_info_df['load_timestamp'] = pd.Timestamp.now()
# for col in list(brawler_info_df.columns)[3:-1]:
#     brawler_info_df[f'{col}'] = brawler_info_df[f'{col}'].astype(float)
# brawler_info_df

,name,Rarity,Class,MovementSpeed,AttackRange,AttackBullets,Reload,AttackSuperCharge,AttackSpeed,AttackWidth,SuperRange,SuperSuperCharge,SuperSpeed,load_timestamp
0,8-BIT,Super Rare,Damage Dealer,580.0,10.00,6.0,1.50,8.480,4500.0,0.33,5.00,NaN,1196.0,2023-02-24 09:29:46.409412
1,AMBER,Legendary,Damage Dealer,720.0,8.33,40.0,0.22,4.550,3500.0,1.33,7.33,6.36,1750.0,2023-02-24 09:29:46.409412
2,ASH,Chromatic,Tank,720.0,4.67,NaN,1.40,16.800,5000.0,1.67,5.00,3.75,1196.0,2023-02-24 09:29:46.409412
3,BARLEY,Rare,Damage Dealer,720.0,7.33,NaN,2.00,17.500,1750.0,NaN,9.33,12.58,1504.0,2023-02-24 09:29:46.409412
4,BEA,Epic,Damage Dealer,720.0,10.00,NaN,0.90,35.600,3255.0,1.00,9.00,2.50,2500.0,2023-02-24 09:29:46.409412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,SQUEAK,Mythic,Damage Dealer,720.0,7.67,NaN,2.10,25.000,4000.0,0.33,8.33,12.50,1600.0,2023-02-24 09:29:46.409412
60,STU,Epic,Assassin,720.0,7.67,2.0,1.50,100.000,3300.0,1.00,2.33,NaN,NaN,2023-02-24 09:29:46.409412
61,SURGE,Chromatic,Damage Dealer,650.0,6.67,NaN,2.00,35.400,3500.0,1.00,2.67,17.00,NaN,2023-02-24 09:29:46.409412
62,TARA,Mythic,Damage Dealer,720.0,8.00,3.0,2.00,8.395,3152.0,0.67,6.67,18.00,1522.0,2023-02-24 09:29:46.409412


In [15]:
# health_attack_df = pd.DataFrame(brawlerDict).T.iloc[:,-2:]
# health_attack_df = health_attack_df.reset_index()
# health_attack_df = health_attack_df.rename(columns = {'index':'name'})

# temp = pd.DataFrame({'power': health_attack_df['Health'].apply(lambda x: list(x.keys())).tolist(), 
#                      'Health': health_attack_df['Health'].apply(lambda x: list(x.values())).tolist(),
#                      'Attack': health_attack_df['Attack'].apply(lambda x: list(x.values())).tolist()})

# health_attack_df = health_attack_df[['name']].join(temp).apply(pd.Series.explode)
# health_attack_df['load_timestamp'] = pd.Timestamp.now()
# health_attack_df['power'] = health_attack_df['power'].astype('int64')
# health_attack_df['Health'] = health_attack_df['Health'].astype('float')
# health_attack_df['Attack'] = health_attack_df['Attack'].astype('float')
# health_attack_df

,name,power,Health,Attack,load_timestamp
0,8-BIT,1,4800.0,320.0,2023-02-24 09:29:46.458710
0,8-BIT,2,5040.0,336.0,2023-02-24 09:29:46.458710
0,8-BIT,3,5280.0,352.0,2023-02-24 09:29:46.458710
0,8-BIT,4,5520.0,368.0,2023-02-24 09:29:46.458710
0,8-BIT,5,5760.0,384.0,2023-02-24 09:29:46.458710
...,...,...,...,...,...
63,TICK,7,2860.0,832.0,2023-02-24 09:29:46.458710
63,TICK,8,2970.0,864.0,2023-02-24 09:29:46.458710
63,TICK,9,3080.0,896.0,2023-02-24 09:29:46.458710
63,TICK,10,3190.0,928.0,2023-02-24 09:29:46.458710


In [16]:
# import psycopg2
# import psycopg2.extras as extras
# from sqlalchemy import create_engine

In [17]:
# PGHOST = 'mj199-database.cfk5zj0ywdiy.us-east-1.rds.amazonaws.com'
# PGDATABASE = 'brawl_stars_db'
# PGUSER = 'postgres'
# PGPASSWORD = 'mj199mj199'

In [18]:
# sa_conn_string = f'postgresql://{PGUSER}:{PGPASSWORD}@{PGHOST}/{PGDATABASE}'
# engine = create_engine(sa_conn_string)
# sa_conn = engine.connect()

In [19]:
# def connect():
    
#     # Set up a connection to the postgres server.
#     conn_string = "host="+ PGHOST +" port="+ "5432" +" dbname="+ PGDATABASE +" user=" + PGUSER \
#                   +" password="+ PGPASSWORD
    
#     conn = psycopg2.connect(conn_string)
#     print("Connected!")

#     # Create a cursor object
#     cursor = conn.cursor()
    
#     return conn, cursor

In [20]:
# conn, cursor = connect()

Connected!


In [21]:
# # Creating brawler_info table
# brawler_info_df.to_sql('brawler_info', con = engine, if_exists = 'append', index=False)
# sql='''select * from "brawler_info";'''

# # Execute SQL Command and commit to DB
# cursor.execute(sql)
# conn.commit()

In [22]:
## cursor.execute('''drop table "brawler_info"''')
## conn.commit()

In [23]:
# # Creating brawler_health_attack table
# health_attack_df.to_sql('brawler_health_attack', con = engine, if_exists = 'append', index=False)
# sql='''select * from "brawler_health_attack";'''

# # Execute SQL Command and commit to DB
# cursor.execute(sql)
# conn.commit()

In [7]:
## cursor.execute('''drop table "brawler_health_attack"''')
## conn.commit()